In [1]:
# Automatically reload code when changes are made
%load_ext autoreload
%autoreload 2
import os
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
from scipy import signal
import pandas as pd
import seaborn as sns
from glob import glob as ls
import sys
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score as sklearn_metric

from utilities import *
from intro_science_donnees import data
from intro_science_donnees import *

In [4]:
#dataset_dir = os.path.join(data.dir, 'ApplesAndBananas')
dataset_dir = 'data'

images = load_images(dataset_dir, "*.jpg")

In [5]:
images[15] = images[15].crop(box=(5, 60, 250, 115))
images[16] = images[16].crop(box=(10, 30, 250, 160))
images[17] = images[17].crop(box=(20, 55, 220, 155))
images[18] = images[18].crop(box=(80, 65, 160, 110))
images[19] = images[19].crop(box=(50, 50, 220, 130))
images[20] = images[20].crop(box=(40, 100, 140, 145))
images[21] = images[21].crop(box=(20, 50, 160, 130))
images[22] = images[22].crop(box=(10, 60, 240, 140))
images[23] = images[23].crop(box=(10, 35, 260, 100))
images[24]= images[24].crop(box=(10, 30, 170, 80))
images[25] = images[25].crop(box=(50, 60, 210, 120))
images[26] = images[26].crop(box=(20, 40, 230, 90))
images[27] = images[27].crop(box=(20, 30, 240, 125))
images[28] = images[28].crop(box=(50, 40, 240, 120))
images[29] = images[29].crop(box=(10, 50, 240, 100))
images[0]=images[0].crop(box=(60,90,170,140))
images[1]=images[1].crop(box=(20,50,230,150))
images[2]=images[2].crop(box=(70,30,240,80))
images[3]=images[3].crop(box=(140,40,230,80))
images[4] = images[4].crop(box=(5, 40, 250, 130))
images[5] = images[5].crop(box=(4, 25, 260, 105))
images[6] = images[6].crop(box=(5, 40, 240, 120))
images[7] = images[7].crop(box=(40, 50, 220, 130))
images[8] = images[8].crop(box=(10, 30, 260, 90))
images[9] = images[9].crop(box=(20, 45, 210, 160))
images[10] = images[10].crop(box=(130, 45, 220, 80))
images[11] = images[11].crop(box=(5, 40, 240, 110))
images[12] = images[12].crop(box=(40, 20, 255, 70))
images[13] = images[13].crop(box=(20, 45, 230, 150))
images[14] = images[14].crop(box=(30, 80, 150, 120))

In [6]:
os.makedirs('clean_data', exist_ok=True)
for name, img in images.items():
    img.save(os.path.join('clean_data', os.path.splitext(name)[0]+".png"))
dataset_dir = 'clean_data'

images = load_images(dataset_dir, "*.png")

In [7]:
def my_foreground_filter(img):
    foreground = foreground_redness_filter(img, theta=.6)
    foreground = invert_if_light_background(foreground)
    foreground = scipy.ndimage.gaussian_filter(foreground, sigma=.05)
    return foreground

In [8]:
def my_preprocessing(img):
    """
    Prétraitement d'une image
    
    - Calcul de l'avant plan
    - Mise en transparence du fond
    - Calcul du centre
    - Recadrage autour du centre
    """
    foreground = my_foreground_filter(img)
    img = transparent_background(img, foreground)
    coordinates = np.argwhere(foreground)
    if len(coordinates) == 0: # Cas particulier: il n'y a aucun pixel dans l'avant plan
        width, height = img.size
        center = (width/2, height/2)
    else:
        center = (np.mean(coordinates[:, 1]), np.mean(coordinates[:, 0]))
    #img = crop_around_center(img, center)
    return img

In [9]:
clean_images = images.apply(my_preprocessing)

clean_sample = list(clean_images[:15]) + list(clean_images[-15:])

In [10]:
df_features = pd.DataFrame({'redness': clean_images.apply(redness),
                            'greenness': clean_images.apply(greenness),
                            'blueness': clean_images.apply(blueness),
                            'elongation': clean_images.apply(elongation),
                            'perimeter': clean_images.apply(perimeter),
                            'surface': clean_images.apply(surface),
                            'blackness' : clean_images.apply(blackness),
                            'espace_milieu' : clean_images.apply(espacemilieu)
                           })


In [11]:
df_features

,redness,greenness,blueness,elongation,perimeter,surface,blackness,espace_milieu
a01.png,10.024741,-5.181165,-4.843575,2.412747,10648,2506,0.544364,0
a02.png,39.165983,9.322763,-48.488746,1.966726,30215,9019,0.630524,0
a03.png,-0.631890,0.891000,-0.259109,3.540294,16022,3211,0.622235,0
a04.png,31.630877,6.858958,-38.489835,2.304610,7444,1574,0.562778,0
a05.png,36.051329,24.934083,-60.985412,2.541169,37585,9254,0.580317,0
a06.png,0.314684,0.079734,-0.394419,3.132720,32615,7525,0.632568,0
a07.png,78.478861,-23.440725,-55.038136,2.894219,23976,5795,0.691809,0
a08.png,53.514957,-12.175214,-41.339744,2.085250,18771,4212,0.707500,0
a09.png,36.094732,-1.796057,-34.298675,4.034134,25044,6492,0.583200,0
a10.png,37.603114,8.450796,-46.053910,1.619910,31640,8607,0.606316,0
